**⚠️ IMPORTANT! ⚠️**

---

**🔓 The Notebook is read-only:** Use **File ▸ Save a copy in Drive** to make it yours, then run cells ✅

---

📚 **This Notebook** is part of the **Input Validation & Guardrails** series:

- [Input Validation](https://colab.research.google.com/drive/1bQwv3o1dBfTlTz4koGF5AXFt3ArnwwtA#copy=true) - Basic input validation
- [Toxicity Detection](https://colab.research.google.com/drive/1cd73fGf-DkyM_tMHrh_eYQ7NWWsn5wgd#copy=true) — Block harmful content
- [PII Detection](https://colab.research.google.com/drive/1hcNw8n8w3oEk2MfGQoG1XuJphfbhcgwM#copy=true) — Protect sensitive information
- [Prompt Injection Detection](https://colab.research.google.com/drive/1eNuATWd2HbuLUvJptiXgmsAgWkgwPVDv#copy=true) — Prevent manipulation attempts
- [Off-Topic Detection](https://colab.research.google.com/drive/1cd73fGf-DkyM_tMHrh_eYQ7NWWsn5wgd#copy=true) — Keep conversations focused

# 🎯 Input Validation & Guardrails

## Off-Topic Detection for LLM Applications

Welcome to this notebook on **Off-Topic Detection** — the final line of defense that ensures your RAG system stays focused on its intended purpose!

Even if a query is safe, non-toxic, and well-formed, it might simply be **irrelevant** to your system's domain. If your RAG system is built to answer questions about "Company HR Policy," it has no business answering questions about "Python coding" or "History of Rome."


## 🚫 Why Off-Topic Detection Matters

Without this guardrail, your system will attempt to retrieve documents for irrelevant queries. Since vector search **always returns something** (even if the match is poor), the LLM will receive random, irrelevant context chunks.

This leads to three major problems:

| Problem | Description | Impact |
|---------|-------------|--------|
| **🤖 Hallucinations** | LLM tries to force an answer using irrelevant context | Incorrect/misleading responses |
| **💸 Resource Waste** | You pay for embedding, retrieval, and generation on unanswerable queries | Wasted costs |
| **🏷️ Brand Risk** | Your specialized bot acts like a general-purpose chatbot | Dilutes product purpose |

### Example Scenario:
```text
User asks: "What is the capital of France?" (to your HR Policy bot)

❌ Without Off-Topic Detection:
   → Vector search returns random HR docs with low similarity
   → LLM receives irrelevant context about vacation policies
   → LLM either hallucinates or gives a confused answer
   → You paid for the full pipeline for a question you can't answer

✅ With Off-Topic Detection:
   → Query flagged as off-topic (low similarity to HR anchors)
   → Immediate response: "I can only help with HR-related questions."
   → No wasted resources, clear user guidance
```

## 📋 Prerequisites

Before we start, make sure you have:
- ✅ A Google Colab environment (you're already here!)
- ✅ Internet connection
- ✅ An OpenAI API Key (we'll guide you through this)


## 🔧 Environment Setup

First, let's install the required libraries.

In [ ]:
# Install the required libraries
!pip install openai sentence-transformers -q

In [ ]:
# Import all required libraries
from openai import OpenAI
from sentence_transformers import SentenceTransformer

print("✅ Libraries imported successfully!")

## 🔑 Secure API Key Management

### Obtaining Your API Key

To get your API key:

1. Go to [Hooloovoo Help Desk - AI Control Center](https://hooloovoo.atlassian.net/servicedesk/customer/portal/48)
2. Navigate to **AI API Keys** section
3. Submit the request (NOTE: Provider should be OpenAI)

### Security Best Practices

**DO:**
- ✅ Store keys as environment variables or in secure storage
- ✅ Use separate keys for development and production
- ✅ Set usage limits and monitoring
- ✅ Rotate keys periodically

**DON'T:**
- ❌ Hard-code keys in your source code
- ❌ Commit keys to version control
- ❌ Share keys publicly or in screenshots
- ❌ Share your keys with others

### Setting Up Your API Key

Run the cell below and paste your API key when prompted (it will be hidden for security).

In [ ]:
# Securely input your API key
import os
from getpass import getpass

# Prompt for API key (input will be hidden)
api_key = getpass("Enter your OpenAI API Key: ")

# Set it as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

print("✅ API key configured successfully!")

## 🧪 Helper Function

Let's create a simple helper function to make API calls easier to read and understand.

In [ ]:
def call_llm(prompt, model="gpt-4.1-nano", temperature=0.7):

    # Build parameters dictionary
    params = {
        "model": model,
        "input": prompt,
    }

    if not model == "gpt-5-codex":
      params["temperature"] = temperature

    response = client.responses.create(**params)
    return response.output_text

print("✅ Helper function defined!")

## 🧭 The Approach: Semantic Boundary Checking

Instead of training a classifier to recognize every possible off-topic category (which is impossible), we use **Similarity-Based Validation**:

1. **Define anchor queries** — A list of "canonical" on-topic example queries
2. **Embed the anchors** — Convert them to vectors in semantic space
3. **Compare incoming queries** — Check cosine similarity against anchors
4. **Apply threshold** — If similarity is too low, reject as off-topic


## 🛠️ Building the Topic Guardrail

Let's create a `TopicGuardrail` class that:
1. Takes a list of on-topic example queries as "anchors"
2. Embeds them into vector space
3. Compares new queries against these anchors using cosine similarity
4. Rejects queries that fall below the similarity threshold

In [ ]:
import numpy as np
from typing import Optional

class TopicGuardrail:
    """
    Semantic boundary checker for off-topic detection.
    Uses cosine similarity against anchor queries to determine relevance.
    """

    def __init__(
        self,
        on_topic_examples: list[str],
        threshold: float = 0.35,
        model_name: str = 'all-MiniLM-L6-v2'
    ):
        """
        Initialize the topic guardrail.

        Args:
            on_topic_examples: List of example queries that ARE in scope
            threshold: Minimum similarity to be considered on-topic (0.0 to 1.0)
            model_name: Sentence transformer model to use
        """
        print(f"Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name)
        self.threshold = threshold
        self.on_topic_examples = on_topic_examples

        # Pre-compute embeddings for all on-topic examples
        print(f"Embedding {len(on_topic_examples)} anchor queries...")
        self.topic_embeddings = self.model.encode(on_topic_examples)

        print("✅ Topic guardrail ready!")

    def check(self, query: str) -> tuple[bool, dict]:
        """
        Check if a query is on-topic.

        Args:
            query: The user's input query

        Returns:
            tuple: (is_on_topic, details)
                - is_on_topic: True if query is within scope
                - details: Dict with similarity scores and matched anchor
        """
        # Embed the incoming query
        query_embedding = self.model.encode([query])[0]

        # Calculate cosine similarity against all anchor embeddings
        # Cosine similarity = dot(A, B) / (norm(A) * norm(B))
        similarities = np.dot(self.topic_embeddings, query_embedding) / (
            np.linalg.norm(self.topic_embeddings, axis=1) *
            np.linalg.norm(query_embedding)
        )

        # Find the best matching anchor
        max_idx = int(np.argmax(similarities))
        max_similarity = float(similarities[max_idx])
        best_match = self.on_topic_examples[max_idx]

        # Determine if on-topic based on threshold
        is_on_topic = max_similarity >= self.threshold

        return is_on_topic, {
            "max_similarity": max_similarity,
            "best_match": best_match,
            "threshold": self.threshold,
            "all_similarities": dict(zip(self.on_topic_examples, similarities.tolist()))
        }

print("✅ TopicGuardrail class defined!")

## 🏢 Example: HR Policy Assistant

Let's create a topic guardrail for an **HR Policy Assistant** that should only answer questions about company policies, benefits, and HR procedures.

In [ ]:
# Define anchor queries for an HR Policy Assistant
hr_anchor_queries = [
    # Vacation & Time Off
    "What is our vacation policy?",
    "How do I request time off?",
    "How many sick days do I get?",

    # Benefits
    "What are the health insurance options?",
    "How does the 401k matching work?",
    "What is the parental leave policy?",

    # Procedures
    "How do I submit an expense report?",
    "What is the dress code?",
    "How do I report harassment?",

    # General HR
    "What are the office hours?",
    "How do I update my direct deposit?",
    "What is the remote work policy?",
]

# Create the topic guardrail
hr_guardrail = TopicGuardrail(
    on_topic_examples=hr_anchor_queries,
    threshold=0.35  # Queries must have at least 35% similarity to be on-topic
)

## ✅ Testing On-Topic Queries

Let's test with queries that SHOULD be allowed — they're related to HR topics.

In [ ]:
# Test with on-topic queries
on_topic_queries = [
    "How many vacation days do I get per year?",
    "Can I work from home on Fridays?",
    "What's the process for requesting maternity leave?",
    "Does the company match 401k contributions?",
    "Where do I submit my travel expenses?",
]

print("Testing ON-TOPIC queries:\n")
print("=" * 70)

for query in on_topic_queries:
    is_on_topic, details = hr_guardrail.check(query)
    status = "✅ ON-TOPIC" if is_on_topic else "❌ OFF-TOPIC"

    print(f"\nQuery: \"{query}\"")
    print(f"Result: {status}")
    print(f"Similarity: {details['max_similarity']:.2f} (threshold: {details['threshold']})")
    print(f"Best match: \"{details['best_match']}\"")

## ❌ Testing Off-Topic Queries

Now let's test with queries that should be REJECTED — they have nothing to do with HR.

In [ ]:
# Test with off-topic queries
off_topic_queries = [
    "What is the capital of France?",
    "How do I write a Python function?",
    "What's the weather like today?",
    "Can you help me with my math homework?",
    "Who won the Super Bowl last year?",
    "What is machine learning?",
]

print("Testing OFF-TOPIC queries:\n")
print("=" * 70)

for query in off_topic_queries:
    is_on_topic, details = hr_guardrail.check(query)
    status = "✅ ON-TOPIC" if is_on_topic else "❌ OFF-TOPIC"

    print(f"\nQuery: \"{query}\"")
    print(f"Result: {status}")
    print(f"Similarity: {details['max_similarity']:.2f} (threshold: {details['threshold']})")
    print(f"Best match: \"{details['best_match']}\"")

## 📊 Visualizing the Similarity Scores

Let's create a visual comparison showing how on-topic vs off-topic queries score against our anchors.

In [ ]:
# Combined visualization
all_test_queries = [
    ("On-topic", "How many vacation days do I get?"),
    ("On-topic", "What's the health insurance deductible?"),
    ("On-topic", "Can I expense my home office setup?"),
    ("Off-topic", "What is the capital of France?"),
    ("Off-topic", "How do I cook pasta?"),
    ("Off-topic", "Explain quantum computing"),
]

print("📊 SIMILARITY SCORE VISUALIZATION")
print("=" * 70)
print(f"Threshold: {hr_guardrail.threshold}")
print("=" * 70)

for category, query in all_test_queries:
    is_on_topic, details = hr_guardrail.check(query)
    score = details["max_similarity"]

    # Create visual bar
    bar_length = int(score * 40)
    threshold_pos = int(hr_guardrail.threshold * 40)

    # Build the bar with threshold marker
    bar = ""
    for i in range(40):
        if i == threshold_pos:
            bar += "|"  # Threshold marker
        elif i < bar_length:
            bar += "█"
        else:
            bar += "░"

    status = "✅" if is_on_topic else "❌"

    print(f"\n[{category}] \"{query[:35]}{'...' if len(query) > 35 else ''}\"")
    print(f"  [{bar}] {score:.2f} {status}")

print("\n" + "=" * 70)
print("Legend: | = threshold, █ = similarity score")

## 🔧 Tuning the Threshold

The threshold is crucial for balancing **coverage** vs **precision**:

| Threshold | Effect | Use Case |
|-----------|--------|----------|
| **Lower (0.25)** | More permissive, allows borderline queries | General-purpose assistants |
| **Medium (0.35)** | Balanced approach | Most production systems |
| **Higher (0.50)** | More strict, only very relevant queries pass | High-precision systems |

Let's see how different thresholds affect our results:

In [ ]:
# Test different thresholds
test_queries = [
    ("How many PTO days do new employees get?", "Clear HR question"),
    ("What's the company policy on side projects?", "Borderline HR question"),
    ("How do I improve my productivity at work?", "General work question"),
    ("What programming languages should I learn?", "Off-topic tech question"),
]

thresholds = [0.25, 0.35, 0.45, 0.55]

print("📈 THRESHOLD SENSITIVITY ANALYSIS")
print("=" * 80)

# Header
print(f"\n{'Query':<40} | ", end="")
for t in thresholds:
    print(f"t={t:<5} | ", end="")
print("\n" + "-" * 80)

for query, description in test_queries:
    # Get the base similarity score
    _, details = hr_guardrail.check(query)
    score = details["max_similarity"]

    # Display query
    display_query = query[:38] + ".." if len(query) > 40 else query
    print(f"{display_query:<40} | ", end="")

    # Check against each threshold
    for t in thresholds:
        status = "✅" if score >= t else "❌"
        print(f"{status} {score:.2f} | ", end="")

    print(f" ({description})")

---

## 🎯 Improving Embedding Accuracy: Fine-Tuning Considerations

While the `all-MiniLM-L6-v2` model works well for general use cases, you may find that it doesn't capture the nuances of your specific domain. Here are strategies to improve accuracy:

### When Embedding-Based Detection Falls Short

| Symptom | Cause | Solution |
|---------|-------|----------|
| Too many false positives | Threshold too high, or model doesn't understand domain | Lower threshold OR fine-tune model |
| Too many false negatives | Threshold too low, or anchors don't cover all topics | Add more anchors OR fine-tune model |
| Borderline queries misclassified | General-purpose embeddings lack domain specificity | Fine-tune on domain data |

### Fine-Tuning Open-Source Embedding Models

If you have labeled data (queries marked as on-topic or off-topic), you can fine-tune open-source models for better accuracy:

**Popular Models for Fine-Tuning:**
- [`sentence-transformers/all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) — Fast, good baseline
- [`sentence-transformers/all-mpnet-base-v2`](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) — Higher quality, 5x slower
- [`BAAI/bge-small-en-v1.5`](https://huggingface.co/BAAI/bge-small-en-v1.5) — Excellent for retrieval tasks
- [`intfloat/e5-small-v2`](https://huggingface.co/intfloat/e5-small-v2) — Strong performance on semantic similarity

**Fine-Tuning Approach (Conceptual):**

```python
# Conceptual example - not executable in this notebook
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# 1. Prepare training data as pairs
train_examples = [
    # (query, anchor, label) - label=1 if similar, 0 if not
    InputExample(texts=["How do I get time off?", "What is our vacation policy?"], label=1.0),
    InputExample(texts=["What's the weather?", "What is our vacation policy?"], label=0.0),
    # ... more examples
]

# 2. Create dataloader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# 3. Define loss function
train_loss = losses.CosineSimilarityLoss(model)

# 4. Fine-tune
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100
)
```

**When to Fine-Tune:**
- You have 500+ labeled query pairs
- Domain-specific vocabulary (legal, medical, technical)
- Current accuracy is below 85% on your test set
- False positive/negative rate is unacceptable for your use case

> *💡 **Tip:***
>
> *Start with more anchors and threshold tuning before investing in fine-tuning. Often, adding 10-20 well-chosen anchor queries can dramatically improve accuracy without the complexity of training.*

---

# 🤖 A Different Approach: LLM-Based Off-Topic Detection

While embedding-based detection is fast and cost-effective, **LLM-based classification** offers superior accuracy for nuanced decisions. Modern small LLMs like `gpt-4.1-nano` can perform sophisticated reasoning about topic relevance at minimal cost.

### Why Use LLM for Off-Topic Detection?

| Advantage | Description |
|-----------|-------------|
| **🎯 Higher Accuracy** | LLMs understand context, intent, and nuance better than cosine similarity |
| **🧠 Reasoning Capability** | Can explain WHY a query is off-topic, useful for debugging |
| **📝 No Anchor Management** | Describe your domain in natural language instead of curating examples |
| **🔄 Easy Updates** | Change the system description without re-embedding anchors |
| **🌐 Handles Edge Cases** | Better at borderline queries that embeddings might misclassify |

### When to Choose LLM Over Embeddings?

| Use Case | Best Approach |
|----------|---------------|
| High volume, low latency (>1000 req/min) | Embeddings |
| High accuracy requirements | LLM |
| Limited anchor examples available | LLM |
| Complex, nuanced domain boundaries | LLM |
| Cost-sensitive production | Embeddings |
| Borderline query handling | LLM |

In [ ]:
import json
import time

class LLMTopicGuardrail:
    """
    LLM-based off-topic detection using fast, efficient classification.
    Uses a small model (gpt-4.1-nano) for cost-effective, accurate decisions.
    """

    def __init__(self, domain_description: str, allowed_topics: list[str]):
        """
        Initialize the LLM topic guardrail.

        Args:
            domain_description: Clear description of what the system handles
            allowed_topics: List of allowed topic categories
        """
        self.domain_description = domain_description
        self.allowed_topics = allowed_topics
        self.model = "gpt-4.1-nano"  # Fast, cheap, accurate for classification

        # Build the classification prompt template
        self.system_prompt = self._build_system_prompt()

    def _build_system_prompt(self) -> str:
        """Build an optimized system prompt for fast classification."""
        topics_str = ", ".join(self.allowed_topics)

        return f"""You are a query classifier. Your task is to determine if a user query is ON-TOPIC or OFF-TOPIC.

DOMAIN: {self.domain_description}

ALLOWED TOPICS: {topics_str}

RULES:
- ON-TOPIC: Query relates to any of the allowed topics
- OFF-TOPIC: Query is unrelated to the domain (general knowledge, other domains, chitchat)

Respond with ONLY valid JSON in this exact format:
{{"decision": "ON-TOPIC" or "OFF-TOPIC", "reason": "brief explanation"}}"""

    def check(self, query: str) -> tuple[bool, dict]:
        """
        Check if a query is on-topic using LLM classification.

        Args:
            query: The user's input query

        Returns:
            tuple: (is_on_topic, details)
        """
        start_time = time.time()

        # Build the prompt
        prompt = f"{self.system_prompt}\n\nQUERY: {query}\n\nJSON:"

        try:
            # Call LLM with low temperature for consistent classification
            response = call_llm(prompt, model=self.model, temperature=0)

            # Parse JSON response
            result = json.loads(response.strip())

            is_on_topic = result.get("decision", "").upper() == "ON-TOPIC"
            reason = result.get("reason", "No reason provided")

        except (json.JSONDecodeError, Exception) as e:
            # Fallback: try to extract decision from raw text
            print(f"⚠️ Error parsing JSON: {e}")
            response_upper = response.upper() if 'response' in dir() else ""
            is_on_topic = "ON-TOPIC" in response_upper and "OFF-TOPIC" not in response_upper
            reason = f"Parse error, inferred from response"

        elapsed_ms = (time.time() - start_time) * 1000

        return is_on_topic, {
            "decision": "ON-TOPIC" if is_on_topic else "OFF-TOPIC",
            "reason": reason,
            "latency_ms": elapsed_ms,
            "model": self.model
        }

print("✅ LLMTopicGuardrail class defined!")

## 🏢 Example: HR Policy Assistant with LLM Detection

Let's create an LLM-based topic guardrail for the same HR Policy Assistant scenario.

In [ ]:
# Create an LLM-based topic guardrail for HR
llm_hr_guardrail = LLMTopicGuardrail(
    domain_description="Company HR Policy Assistant that answers questions about employee policies, benefits, procedures, and workplace guidelines.",
    allowed_topics=[
        "vacation and time off",
        "health insurance and benefits",
        "401k and retirement",
        "parental leave",
        "expense reports",
        "dress code",
        "harassment reporting",
        "remote work policies",
        "office hours",
        "payroll and compensation"
    ]
)

print("✅ LLM HR Topic Guardrail created!")
print(f"   Model: {llm_hr_guardrail.model}")
print(f"   Topics: {len(llm_hr_guardrail.allowed_topics)} categories")

## 🧪 Testing LLM-Based Detection

Let's test the LLM guardrail with a variety of queries and see how it performs.

In [ ]:
# Test queries - mix of on-topic and off-topic
test_queries = [
    # Clear on-topic
    ("How many vacation days do new employees get?", "On-topic"),
    ("What's the process for requesting parental leave?", "On-topic"),
    ("Does the company match 401k contributions?", "On-topic"),

    # Borderline / nuanced
    ("Can I work from a coffee shop?", "Borderline - remote work related"),
    ("What should I wear to a client meeting?", "Borderline - dress code related"),
    ("How do I improve my work-life balance?", "Borderline - could be HR or general"),

    # Clear off-topic
    ("What is the capital of France?", "Off-topic"),
    ("How do I write a Python function?", "Off-topic"),
    ("What's the best pizza topping?", "Off-topic"),
]

print("🤖 LLM-BASED OFF-TOPIC DETECTION")
print("=" * 70)

total_latency = 0
for query, expected in test_queries:
    is_on_topic, details = llm_hr_guardrail.check(query)
    total_latency += details["latency_ms"]

    status = "✅ ON-TOPIC" if is_on_topic else "❌ OFF-TOPIC"

    print(f"\n📝 Query: \"{query}\"")
    print(f"   Expected: {expected}")
    print(f"   Result: {status}")
    print(f"   Reason: {details['reason']}")
    print(f"   Latency: {details['latency_ms']:.0f}ms")

avg_latency = total_latency / len(test_queries)
print(f"\n" + "=" * 70)
print(f"📊 Average latency: {avg_latency:.0f}ms per query")

### ⚡️ Latency optimizations

This optimized guardrail focuses on **reducing tokens, avoiding retries, and enabling cache hits**:

- **System prompt kept separate & stable** (sent as a system message, not concatenated into a big string). This reduces prompt churn and is better for caching.
- **Structured Outputs (schema-enforced)** so the model *must* return valid JSON → no `json.loads()` failures, no fallback logic, no hidden retries.
- **Optimize Response** so that model returns `bool` not `string` for `is_on_topic` field. Remove `reason` field (this can be "enabled" during testing).
- **Hard cap on output length** via `max_output_tokens` (we only need `decision` + optional short `reason`).
- **Disable storage** (`store=False`) for slightly leaner requests (and better privacy defaults).

In [ ]:
import time
from typing import Literal, Optional, Tuple, Dict, Any, List

from openai import OpenAI
from pydantic import BaseModel, Field


class TopicDecision(BaseModel):
    is_on_topic: bool
    # Make reason optional so you can run "ultra-fast mode" with fewer output tokens.
    reason: Optional[str] = Field(default=None, max_length=120)


class OptimizedLLMTopicGuardrail:
    """
    Optimized LLM-based off-topic detection for low latency.

    Key knobs:
      - system prompt as a stable system message (better caching, less prompt noise)
      - structured outputs (no JSON parsing drama)
      - max_output_tokens kept tiny
      - prompt caching enabled (prompt_cache_key + prompt_cache_retention)
    """

    def __init__(
        self,
        domain_description: str,
        allowed_topics: List[str],
        *,
        client: Optional[OpenAI] = None,
        model: str = "gpt-4.1-nano",
        temperature: float = 0.0,
        max_output_tokens: int = 40,
        include_reason: bool = True,
        store: bool = False,
    ):
        self.client = client or OpenAI()
        self.model = model
        self.temperature = temperature
        self.max_output_tokens = max_output_tokens
        self.include_reason = include_reason
        self.store = store

        # Precompute a compact, stable system prompt once (important for cache hit rate).
        topics = "\n".join(f"- {t.strip()}" for t in allowed_topics if t and t.strip())
        self.system_prompt = (
            "You are a strict topic classifier.\n"
            "Return ONLY the JSON object matching the schema.\n"
            "Decide on_topic is true if the query clearly relates to the domain/topics; otherwise false.\n"
            f"DOMAIN: {domain_description.strip()}\n"
            f"TOPICS:\n{topics}\n"
        )

    def check(self, query: str, *, cache_key: Optional[str] = None) -> Tuple[bool, Dict[str, Any]]:
        """
        Args:
            query: user input
            cache_key: stable identifier to improve cache hit rates (e.g., hashed user/org id)

        Returns:
            (is_on_topic, details_dict)
        """
        start = time.time()

        # Keep the user message tiny. The heavy, stable prefix is in the system message.
        input_messages = [
            {"role": "system", "content": self.system_prompt},
            {
                "role": "user",
                "content": (
                    f"QUERY: {query}\n"
                    + ("Include a short reason." if self.include_reason else "No reason.")
                ),
            },
        ]

        try:
            # Structured outputs: no JSON parsing, no retry loops for formatting.
            # max_output_tokens: hard cap on generated tokens.
            resp = self.client.responses.parse(
                model=self.model,
                input=input_messages,
                text_format=TopicDecision,
                temperature=self.temperature,
                max_output_tokens=self.max_output_tokens,
                store=self.store
            )

            parsed: TopicDecision = resp.output_parsed

            latency_ms = (time.time() - start) * 1000
            usage = getattr(resp, "usage", None)

            return parsed.is_on_topic, {
                "reason": parsed.reason or "",
                "latency_ms": latency_ms,
                "model": self.model,
            }

        except Exception as e:
            # Fail-safe behavior for a guardrail: treat as OFF_TOPIC (or route to stricter path).
            latency_ms = (time.time() - start) * 1000
            return False, {
                "reason": f"classification_error: {type(e).__name__}",
                "latency_ms": latency_ms,
                "model": self.model,
            }

Let's do another test, first create new guardrail object:

In [ ]:
optimized_llm_hr_guardrail = OptimizedLLMTopicGuardrail(
    domain_description="Company HR Policy Assistant that answers questions about employee policies, benefits, procedures, and workplace guidelines.",
    allowed_topics=[
        "vacation and time off",
        "health insurance and benefits",
        "401k and retirement",
        "parental leave",
        "expense reports",
        "dress code",
        "harassment reporting",
        "remote work policies",
        "office hours",
        "payroll and compensation"
    ],
    client=client,
    include_reason=False  # Disable include reason during "production". Enable during testing, good while tweaking the prompt, temperature or model choice.
)

print("✅ LLM HR Topic Guardrail created!")
print(f"   Model: {llm_hr_guardrail.model}")
print(f"   Topics: {len(llm_hr_guardrail.allowed_topics)} categories")

In [ ]:
# Test queries - mix of on-topic and off-topic
test_queries = [
    # Clear on-topic
    ("How many vacation days do new employees get?", "On-topic"),
    ("What's the process for requesting parental leave?", "On-topic"),
    ("Does the company match 401k contributions?", "On-topic"),

    # Borderline / nuanced
    ("Can I work from a coffee shop?", "Borderline - remote work related"),
    ("What should I wear to a client meeting?", "Borderline - dress code related"),
    ("How do I improve my work-life balance?", "Borderline - could be HR or general"),

    # Clear off-topic
    ("What is the capital of France?", "Off-topic"),
    ("How do I write a Python function?", "Off-topic"),
    ("What's the best pizza topping?", "Off-topic"),
]

print("🤖 LLM-BASED OFF-TOPIC DETECTION")
print("=" * 70)

total_latency = 0
for query, expected in test_queries:
    is_on_topic, details = llm_hr_guardrail.check(query)
    total_latency += details["latency_ms"]

    status = "✅ ON-TOPIC" if is_on_topic else "❌ OFF-TOPIC"

    print(f"\n📝 Query: \"{query}\"")
    print(f"   Expected: {expected}")
    print(f"   Result: {status}")
    print(f"   Reason: {details['reason']}")
    print(f"   Latency: {details['latency_ms']:.0f}ms")

avg_latency = total_latency / len(test_queries)
print(f"\n" + "=" * 70)
print(f"📊 Average latency: {avg_latency:.0f}ms per query")

## ⚡ Performance Comparison: Embeddings vs LLM

Let's compare both approaches on the same set of queries to see the trade-offs.

In [ ]:
# Compare both approaches
comparison_queries = [
    "How do I request time off for a medical appointment?",
    "What benefits are available for part-time employees?",
    "Can I expense my home office furniture?",
    "What programming language should I learn?",
    "Who won the World Cup?",
    "Is there a policy about bringing pets to the office?",  # Borderline
]

print("⚡ EMBEDDING vs LLM COMPARISON")
print("=" * 75)
print(f"{'Query':<45} {'Embedding':<15} {'LLM':<15}")
print("-" * 75)

embedding_times = []
llm_times = []

for query in comparison_queries:
    # Embedding approach
    start = time.time()
    emb_on_topic, emb_details = hr_guardrail.check(query)
    emb_time = (time.time() - start) * 1000
    embedding_times.append(emb_time)

    # LLM approach
    llm_on_topic, llm_details = optimized_llm_hr_guardrail.check(query)
    llm_times.append(llm_details["latency_ms"])

    # Format results
    emb_result = f"{'✅' if emb_on_topic else '❌'} ({emb_details['max_similarity']:.2f})"
    llm_result = f"{'✅' if llm_on_topic else '❌'}"

    display_query = query[:43] + ".." if len(query) > 45 else query
    print(f"{display_query:<45} {emb_result:<15} {llm_result:<15}")

print("-" * 75)
print(f"{'Average Latency:':<45} {sum(embedding_times)/len(embedding_times):.0f}ms{'':<9} {sum(llm_times)/len(llm_times):.0f}ms")

## 🎯 Choosing the Right Approach

Based on our comparison, here's a decision framework:

| Factor | Embedding-Based | LLM-Based |
|--------|-----------------|-----------|
| **Latency** | ~10-50ms | ~500-1500ms |
| **Cost per query** | ~\$0.00001 (compute only) | ~\$0.0001-0.001 |
| **Accuracy on clear cases** | Excellent | Excellent |
| **Accuracy on borderline** | Good | Excellent |
| **Explainability** | Limited (similarity score) | High (reasoning provided) |
| **Setup effort** | Requires anchor curation | Just describe domain |
| **Maintenance** | Update anchors manually | Update description |

### Recommended Patterns:

**Pattern 1: Embedding-First (Cost Optimized)**
```text
User Query → Embedding Check → If borderline → LLM Check → Decision
```

**Pattern 2: LLM-Only (Accuracy Optimized)**
```text
User Query → LLM Check → Decision
```

**Pattern 3: Parallel (Latency + Accuracy)**
```text
User Query → [Embedding Check || LLM Check] → Combine Results → Decision
```

In [ ]:
def hybrid_topic_check(
    query: str,
    embedding_guardrail: TopicGuardrail,
    llm_guardrail: LLMTopicGuardrail,
    borderline_threshold: tuple = (0.30, 0.50)
) -> tuple[bool, dict]:
    """
    Hybrid approach: Use embeddings first, escalate to LLM for borderline cases.

    This pattern optimizes for both cost and accuracy:
    - Clear on-topic (similarity > 0.50): Accept immediately
    - Clear off-topic (similarity < 0.30): Reject immediately
    - Borderline (0.30-0.50): Use LLM for final decision

    Args:
        query: User query to check
        embedding_guardrail: Pre-configured embedding guardrail
        llm_guardrail: Pre-configured LLM guardrail
        borderline_threshold: (low, high) thresholds for borderline zone
    """
    low_thresh, high_thresh = borderline_threshold

    # Step 1: Fast embedding check
    start = time.time()
    emb_on_topic, emb_details = embedding_guardrail.check(query)
    emb_time = (time.time() - start) * 1000

    similarity = emb_details["max_similarity"]

    # Step 2: Decide based on confidence
    if similarity >= high_thresh:
        # High confidence on-topic - no need for LLM
        return True, {
            "method": "embedding",
            "decision": "ON-TOPIC",
            "reason": f"High similarity ({similarity:.2f}) to '{emb_details['best_match']}'",
            "similarity": similarity,
            "latency_ms": emb_time,
            "llm_used": False
        }

    elif similarity < low_thresh:
        # High confidence off-topic - no need for LLM
        return False, {
            "method": "embedding",
            "decision": "OFF-TOPIC",
            "reason": f"Low similarity ({similarity:.2f}) - not related to domain",
            "similarity": similarity,
            "latency_ms": emb_time,
            "llm_used": False
        }

    else:
        # Borderline case - escalate to LLM
        llm_on_topic, llm_details = llm_guardrail.check(query)
        total_time = emb_time + llm_details["latency_ms"]

        return llm_on_topic, {
            "method": "hybrid (escalated to LLM)",
            "decision": llm_details["decision"],
            "reason": llm_details["reason"],
            "similarity": similarity,
            "latency_ms": total_time,
            "llm_used": True
        }

print("✅ Hybrid topic check function defined!")

# Cell Type: Code
# Test the hybrid approach
hybrid_test_queries = [
    "What is our vacation policy?",           # Clear on-topic
    "What is the capital of France?",         # Clear off-topic
    "Can I bring my dog to work sometimes?",  # Borderline
    "How do I handle stress at work?",        # Borderline
    "What's the dress code for casual Friday?", # On-topic
]

print("🔀 HYBRID APPROACH RESULTS")
print("=" * 75)

for query in hybrid_test_queries:
    is_on_topic, details = hybrid_topic_check(query, hr_guardrail, llm_hr_guardrail)

    status = "✅ ON-TOPIC" if is_on_topic else "❌ OFF-TOPIC"
    llm_indicator = "🤖" if details["llm_used"] else "📊"

    print(f"\n{llm_indicator} Query: \"{query}\"")
    print(f"   Decision: {status}")
    print(f"   Method: {details['method']}")
    print(f"   Reason: {details['reason']}")
    print(f"   Latency: {details['latency_ms']:.0f}ms")

print("\n" + "=" * 75)
print("Legend: 📊 = Embedding only, 🤖 = LLM escalation")

## 🎓 Interactive Demo: Test Your Own Queries

Try modifying the query below to see how the HR guardrail responds!

In [ ]:
# 🎯 TRY IT YOURSELF!
# Modify this query and run the cell

your_query = "How do I request a day off for a doctor's appointment?"

print("=" * 60)
print("YOUR QUERY ANALYSIS")
print("=" * 60)

is_on_topic, details = hr_guardrail.check(your_query)

print(f"\n📝 Query: \"{your_query}\"")
print(f"\n📊 Results:")
print(f"   Status: {'✅ ON-TOPIC' if is_on_topic else '❌ OFF-TOPIC'}")
print(f"   Similarity Score: {details['max_similarity']:.3f}")
print(f"   Threshold: {details['threshold']}")
print(f"   Best Matching Anchor: \"{details['best_match']}\"")

# Show top 3 most similar anchors
print(f"\n🔍 Top 3 Most Similar Anchors:")
sorted_sims = sorted(
    details['all_similarities'].items(),
    key=lambda x: x[1],
    reverse=True
)[:3]

for i, (anchor, sim) in enumerate(sorted_sims, 1):
    print(f"   {i}. \"{anchor}\" → {sim:.3f}")

print("\n" + "=" * 60)

## 📚 Summary: Off-Topic Detection Approaches

| Approach | Best For | Latency | Cost | Accuracy |
|----------|----------|---------|------|----------|
| **Embedding-Based** | High volume, cost-sensitive | ~10-50ms | Very Low | Good |
| **LLM-Based** | High accuracy needs, complex domains | ~100-500ms | Low | Excellent |
| **Hybrid** | Balance of cost and accuracy | ~10-500ms | Optimized | Excellent |

### Key Takeaways:

1. **Start with embeddings** — Fast, cheap, and effective for most cases
2. **Add LLM for borderline cases** — Gets the best of both worlds
3. **Consider fine-tuning** — If embedding accuracy isn't sufficient
4. **Use gpt-4.1-nano** — Fast and cheap for classification tasks
5. **Design clear domain descriptions** — Better prompts = better LLM accuracy